In [3]:
import pandas as pd
!pip install sqlalchemy

from sqlalchemy import create_engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 1.3 MB/s eta 0:00:00a 0:00:010m

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install psycopg2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.7/385.7 kB 1.4 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-macosx_10_9_universal2.whl size=243889 sha256=07355b86a69edf02202651535fda8a59b8255ab8ad58a4828e29004aae84f65a
  Stored in directory: /Users/dennismain/Library/Caches/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [7]:
df = pd.read_csv("green_tripdata_2019-10.csv.gz", low_memory= False)
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2019-10-01 00:26:02,2019-10-01 00:39:58,N,1.0,112,196,1.0,5.88,18.0,0.50,0.5,0.00,0.0,NaN,0.3,19.30,2.0,1.0,0.0
1,1.0,2019-10-01 00:18:11,2019-10-01 00:22:38,N,1.0,43,263,1.0,0.80,5.0,3.25,0.5,0.00,0.0,NaN,0.3,9.05,2.0,1.0,0.0
2,1.0,2019-10-01 00:09:31,2019-10-01 00:24:47,N,1.0,255,228,2.0,7.50,21.5,0.50,0.5,0.00,0.0,NaN,0.3,22.80,2.0,1.0,0.0
3,1.0,2019-10-01 00:37:40,2019-10-01 00:41:49,N,1.0,181,181,1.0,0.90,5.5,0.50,0.5,0.00,0.0,NaN,0.3,6.80,2.0,1.0,0.0
4,2.0,2019-10-01 00:08:13,2019-10-01 00:17:56,N,1.0,97,188,1.0,2.52,10.0,0.50,0.5,2.26,0.0,NaN,0.3,13.56,1.0,1.0,0.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 476386 entries, 0 to 476385
Data columns (total 20 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   VendorID               387007 non-null  float64
 1   lpep_pickup_datetime   476386 non-null  object 
 2   lpep_dropoff_datetime  476386 non-null  object 
 3   store_and_fwd_flag     387007 non-null  object 
 4   RatecodeID             387007 non-null  float64
 5   PULocationID           476386 non-null  int64  
 6   DOLocationID           476386 non-null  int64  
 7   passenger_count        387007 non-null  float64
 8   trip_distance          476386 non-null  float64
 9   fare_amount            476386 non-null  float64
 10  extra                  476386 non-null  float64
 11  mta_tax                476386 non-null  float64
 12  tip_amount             476386 non-null  float64
 13  tolls_amount           476386 non-null  float64
 14  ehail_fee              0 non-null   

In [9]:
df["lpep_dropoff_datetime"] = pd.to_datetime(df["lpep_dropoff_datetime"])
df["lpep_pickup_datetime"] = pd.to_datetime(df["lpep_pickup_datetime"])

In [10]:
df.drop(columns="ehail_fee", inplace= True)

In [14]:
# Database connection parameters
db_user = 'postgres'
db_password = 'postgres'
db_host = 'localhost'
db_port = '5433'
db_name = 'ny_taxi'

In [15]:
# create a connection string
connection_string = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}'
engine = create_engine(connection_string)

In [16]:
engine.connect()

In [17]:
print(pd.io.sql.get_schema(df, name="green_taxi_data", con= engine))


CREATE TABLE green_taxi_data (
	"VendorID" FLOAT(53), 
	lpep_pickup_datetime TIMESTAMP WITHOUT TIME ZONE, 
	lpep_dropoff_datetime TIMESTAMP WITHOUT TIME ZONE, 
	store_and_fwd_flag TEXT, 
	"RatecodeID" FLOAT(53), 
	"PULocationID" BIGINT, 
	"DOLocationID" BIGINT, 
	passenger_count FLOAT(53), 
	trip_distance FLOAT(53), 
	fare_amount FLOAT(53), 
	extra FLOAT(53), 
	mta_tax FLOAT(53), 
	tip_amount FLOAT(53), 
	tolls_amount FLOAT(53), 
	improvement_surcharge FLOAT(53), 
	total_amount FLOAT(53), 
	payment_type FLOAT(53), 
	trip_type FLOAT(53), 
	congestion_surcharge FLOAT(53)
)




In [18]:
df.to_sql(
    'green_tripdata',
    engine,
    if_exists='replace'
)

386

In [19]:
df_zone = pd.read_csv("taxi_zone_lookup.csv")
df_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [20]:
df_zone.to_sql(
    'taxi_zone_lookup',
    engine,
    if_exists='replace'
)

265